In [183]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

path = './datasets/kc_house_data.csv'
date_columns = ['date', 'yr_built', 'yr_renovated']
pd.options.display.float_format = '{:,.2f}'.format
mydateparser = lambda x: pd.to_datetime(x, errors='coerce')

In [240]:
# FUNCTION DECLARATIONS
def load_data(path, date_columns, date_parser):
    return pd.read_csv(path, parse_dates=date_columns, date_parser=date_parser)

def convert_sqft_to_m2(data):
    data['m2_lot'] = data['sqft_lot'] * 0.092903
    data['m2_above'] = data['sqft_above'] * 0.092903
    data['m2_basement'] = data['sqft_basement'] * 0.092903
    data['m2_living'] = data['sqft_living'] * 0.092903
    data['m2_living15'] = data['sqft_living15'] * 0.092903
    data['m2_lot15'] = data['sqft_lot15'] * 0.092903
    data = data.drop(['sqft_lot', 'sqft_above', 'sqft_basement',
                     'sqft_living', 'sqft_living15', 'sqft_lot15'], axis=1)
    data['price_m2'] = data['price'] / data['m2_lot']
    return data

def remove_outliers(data, ignore_columns=[], verbose=False):
    z = np.abs(stats.zscore(data.select_dtypes('number')))
    z[ignore_columns] = 0
    filtered = (z < 3).all(axis=1)
    if verbose:
        for column in z.columns:
            if column not in ignore_columns:
            #    print(f'{column} Original Unique Count: {data[column].nunique()}')
            #   print(f'{column} Filtered Unique Count: {data[z[column] > 3][column].nunique()}')
               print(f'{column} Original Unique Count: {data[column].nunique()} Values: {data[column].sort_values().unique()}')
               print(f'{column} Filtered Unique Count: {data[z[column] > 3][column].nunique()} Values: {data[z[column] > 3][column].sort_values().unique()}')
               print()
    return data[filtered]



In [103]:
data = load_data(path, date_columns, mydateparser)
data = convert_sqft_to_m2(data)

In [241]:
x = remove_outliers(data, ignore_columns=[
                'id', 'waterfront', 'view', 'condition', 'grade', 'zipcode', 'lat', 'long'], verbose=True)



price Original Unique Count: 4028 Values: [  75000.   78000.   80000. ... 6885000. 7062500. 7700000.]
price Filtered Unique Count: 226 Values: [1646000. 1648000. 1650000. 1651000. 1655000. 1660000. 1662000. 1665000.
 1670000. 1675000. 1679000. 1680000. 1681000. 1688000. 1690000. 1691000.
 1695000. 1697000. 1698000. 1698890. 1699000. 1699990. 1700000. 1702500.
 1705000. 1710000. 1712500. 1712750. 1715000. 1720000. 1727000. 1728000.
 1730000. 1735000. 1738000. 1740000. 1749000. 1750000. 1755000. 1760000.
 1762000. 1765000. 1769000. 1770000. 1775000. 1776000. 1780000. 1785000.
 1789950. 1795000. 1799000. 1800000. 1802750. 1810000. 1815000. 1820000.
 1822500. 1824100. 1825000. 1830000. 1835000. 1839900. 1850000. 1851000.
 1855000. 1862000. 1865000. 1870000. 1875000. 1880000. 1881580. 1886700.
 1890000. 1895000. 1898000. 1899000. 1900000. 1901000. 1905000. 1910000.
 1920000. 1925000. 1928000. 1940000. 1945000. 1950000. 1955000. 1959000.
 1960000. 1965000. 1965220. 1970000. 1975000. 1980000.

In [112]:
data.iloc[21608,:]

id                        263000018
date            2014-05-21 00:00:00
price                    360,000.00
bedrooms                          3
bathrooms                      2.50
floors                         3.00
waterfront                        0
view                              0
condition                         3
grade                             8
yr_built        2009-01-01 00:00:00
yr_renovated                    NaT
zipcode                       98103
lat                           47.70
long                        -122.35
m2_lot                       105.07
m2_above                     142.14
m2_basement                    0.00
m2_living                    142.14
m2_living15                  142.14
m2_lot15                     140.19
price_m2                   3,426.18
Name: 21608, dtype: object

In [78]:
# H1: Imóveis que possuem vista para água, são 30% mais caros, na média.
waterfront_mean = data.loc[data['waterfront'] == 1]['price'].mean()
no_waterfront_mean = data.loc[data['waterfront'] == 0]['price'].mean()
h1 = waterfront_mean / no_waterfront_mean
h1


3.126391696351833